---
FIRST

---

In [38]:
# read it in to inspect it
with open('tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [39]:
print("length of dataset in characters: ", len(text))
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
print(chars[1])

length of dataset in characters:  1115394

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
 


In [40]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [41]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [42]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

---
We now need to make characters into spike trains

---

In [43]:
import numpy as np

def generate_spike_trains(input_data, U_max, U_min, N_sample):
    """
    Generate spike trains using Poisson encoding.
    
    Parameters:
        input_data (numpy.ndarray): Normalized input data (e.g., between U_min and U_max).
        U_max (float): Maximum value of the input range.
        U_min (float): Minimum value of the input range.
        N_sample (int): Number of spike samples.
    
    Returns:
        spike_trains (numpy.ndarray): Generated spike trains (0s and 1s).
    """
    # Calculate average spike interval
    h_k = N_sample * (U_max - input_data) / (U_max - U_min)
    
    # Generate intervals using Poisson distribution
    intervals = np.random.poisson(h_k)
    
    # Generate spike trains
    spike_trains = np.zeros((len(input_data), N_sample))
    for i, interval in enumerate(intervals):
        spike_indices = np.cumsum(np.random.choice(np.arange(1, N_sample + 1), interval, replace=False))
        spike_indices = spike_indices[spike_indices < N_sample]
        spike_trains[i, spike_indices] = 111
    
    return spike_trains


# Example input data
input_data = np.array([0.1, 0.4, 0.7, 0.9])  # Normalized input
U_max = 1.0
U_min = 0.0
N_sample = 100  # Spike sampling times

# Generate spike trains
spikes = generate_spike_trains(input_data, U_max, U_min, N_sample)
print("Generated Spike Trains:")
# print(spikes)


Generated Spike Trains:


In [44]:
# SIMPLEST ENCODING
# each character activate 1 neuron, fixed

def char_to_spike_train(char):
    """
    One hot encoding of a character
    
    Parameters: 
        char (string): the character to encode
    
    Returns:
        np.array of dimensions (N_sam, 65)   
    """
    spikes = torch.zeros((1, 65))
    spikes[0, char] = 1

    return spikes

print(char_to_spike_train(0))

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


---
Create the reservoire

N_res is the number of neurons in the reservoir

---

In [45]:
import numpy as np
from scipy.sparse import random
from scipy.sparse.linalg import eigs  # For sparse eigenvalue computation

# Parameters
N_res = 1000  # Size of the reservoir (N_res x N_res)
eta = 0.1    # Sparsity degree (10% non-zero entries)

def uniform_rvs(size):
    return np.random.uniform(low=-1.0, high=1.0, size=size)
# Generate a sparse matrix with uniform random values in [-1, 1]
W = random(N_res, N_res, density=eta, format='csr', data_rvs=uniform_rvs)

# Convert to a dense matrix (optional, for visualization or further processing)
W_dense = W.toarray()

#Calculate the max eigenvalue and find the max
max_eigenvalue = eigs(W, k=1, which='LM', return_eigenvectors=False)[0].real

# Normalize W to get W_res
rho = 0.9
W_res = rho * (W / max_eigenvalue)

---
BATCH EXAMPLE

---

In [46]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

# for b in range(batch_size): # batch dimension
#     for t in range(block_size): # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [36]:
def tensor_to_one_hot(input_tensor, num_classes=65):
    batch_size, seq_len = input_tensor.shape

    # Create a zero tensor of shape (batch_size, seq_len, num_classes)
    one_hot_tensor = torch.zeros((batch_size, seq_len, num_classes), dtype=torch.float32)

    # Fill the one-hot tensor
    for i in range(batch_size):
        for j in range(seq_len):
            one_hot_tensor[i, j, input_tensor[i, j]] = 1

    return one_hot_tensor

# Example usage
input_tensor = torch.tensor([[24, 43, 58,  5, 57,  1, 46, 43],
                              [44, 53, 56,  1, 58, 46, 39, 58],
                              [52, 58,  1, 58, 46, 39, 58,  1],
                              [25, 17, 27, 10,  0, 21,  1, 54]])

one_hot_encoded_tensor = tensor_to_one_hot(input_tensor)
print(one_hot_encoded_tensor.shape)


torch.Size([4, 8, 65])


In [49]:
for b in range(batch_size):
    for t in range(block_size):
        one_hot_encoded_tensor

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


---
Trainable output layer

---

In [ ]:
import torch
import torch.nn as nn

class SingleLayerNN(nn.Module):
    def __init__(self, input_size=1000, output_size=65):
        super(SingleLayerNN, self).__init__()
        # Define a single linear layer
        self.linear = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        # Apply the linear transformation
        output = self.linear(x)
        return output

# Create the model
model = SingleLayerNN(input_size=1000, output_size=65)

# Forward pass
output = model(xb)
print("Output shape:", output.shape)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x8 and 1000x65)

---
PASS

---

In [ ]:
"""
    
"""